In [1]:
#by RPadilla
print("Viva el equip_F en el simulador :) ")

Viva el equip_F en el simulador :) 


In [13]:
# import libraries 
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import mysql.connector
from mysql.connector import Error

## Downloading data
### Connection to MySQL DB and Data Extraction

In [15]:
# name of database
DB = 'Equip_F'

# sprint name to add to data filename
SPRINT = 'sprint_1'

#### Setting Up the `.env` File
Before running the notebook, you will need to create a `.env` file in the notebook folder to provide your MySQL credentials. File should contain variables:
- `MYSQL_USER`=Your MySQL username
- `MYSQL_PASSWORD`=Your MySQL password
- `HOST`=The host where your MySQL server is running

In [29]:
# Automatically find the .env file
dotenv_path = find_dotenv()

# If found, load the .env file
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    print(".env file not found")
# configuration for MySQL connection
config = {
  'user': os.getenv('MYSQL_USER'),
  'password': os.getenv('MYSQL_PASSWORD'),
  'host': os.getenv('HOST'),
  'database': DB,
  'raise_on_warnings': True
}


In [31]:
# define function to connect to MySQL and download data
def get_data(db_name, config):

    """
    Input: db_name - string, database name to fetch data from.
    config - dict, configuration for MySQL connection
    Connects to MySQL and fetches data from all tables.
    Output: 1) dict or pd.DataFrame 
    (dictionary of DataFrames if multiple tables are fetched, 
    otherwise a single DataFrame)
    2) list with tabel names
    """

    try:
        cnx = mysql.connector.connect(**config)
        cursor = cnx.cursor(dictionary=True)

        cursor.execute('show tables;')
        tables = cursor.fetchall()

        data = {}
        table_names = []
        for table in tables:
            table_name = list(table.values())[0]
            table_names.append(table_name)
            
            query = f"select * from {table_name}"
            cursor.execute(query)
            records = cursor.fetchall()
            print(f"Total rows in table {table_name}: {cursor.rowcount}")
            data[table_name] = pd.DataFrame(records)

        return data if len(table_names) > 1 else data[table_names[0]], table_names

    except mysql.connector.Error as error:
        print(f"Failed to get record from MySQL table: {error}")

    finally:
        if cnx.is_connected():
            cursor.close()
            cnx.close()
            print("MySQL connection is closed")

In [33]:
# fetch data from MySQL and store in dictionary
data, tables = get_data(DB, config)

Total rows in table Tourist_Accommodation: 7001
MySQL connection is closed


In [7]:
# save DataFrame to a .csv file in "Data" folder
data.to_csv(f'../Data/data_{SPRINT}.csv', index=False)